In [205]:
import numpy as np
import time
import pandas as pd
from IPython.display import display
import nba_inference_utils as niu
from ccb_model import BootstrapCalibratedClassifier

train_cols = ['Opp_Elo', 'Opp_Momentum', 'SPREAD_LINE_MOVEMENT_1', 
              'SPREAD_LINE_MOVEMENT_2', 'SPREAD_LINE_MOVEMENT_3', 
                'TOTAL_LINE_MOVEMENT_1', 'TOTAL_LINE_MOVEMENT_2',
                'TOTAL_LINE_MOVEMENT_3', 'CREW', 'Opp_Avg_3_game_DEFF',
                'Opp_Avg_5_game_DEFF', 'Opp_Season_Avg_DEFF',
                'Opp_Avg_3_game_OEFF', 'Opp_Avg_5_game_OEFF',
                'Opp_Season_Avg_OEFF', 'Opp_Avg_3_game_PACE',
                'Opp_Avg_5_game_PACE', 'Opp_Season_Avg_PACE',
                'Opp_Avg_3_game_POSS', 'Opp_Avg_5_game_POSS',
                'Opp_Season_Avg_POSS', 'Avg_3_game_DEFF',
                'Avg_5_game_DEFF', 'Season_Avg_DEFF',
                'Avg_3_game_OEFF', 'Avg_5_game_OEFF',
                'Season_Avg_OEFF', 'Avg_3_game_PACE',
                'Avg_5_game_PACE', 'Season_Avg_PACE',
                'Avg_3_game_POSS', 'Avg_5_game_POSS',
                'Season_Avg_POSS', 'CLOSING_SPREAD',
                'CLOSING_TOTAL', 'MONEYLINE', 'Avg_3_game_PTS',
                'Avg_5_game_PTS', 'Season_Avg_PTS', 'Last_ML_1',
                'Last_ML_2', 'Last_ML_3', 'VENUE', 'TEAM', 'Opponent',
                'Win_Loss_Diff', 'HOME TEAM WIN%', 'HOME TEAM POINTS DIFFERENTIAL',
                'TOTAL POINTS PER GAME', 'CALLED FOULS PER GAME',
                'FOUL% AGAINST ROAD TEAMS', 'FOUL% AGAINST HOME TEAMS',
                'FOUL DIFFERENTIAL (Against Road Team) - (Against Home Team)',
                'Elo_Rating', 'Momentum', 'MAIN REF', 'TEAM_REST_DAYS']

today_map_features = ['TEAM', 'Opponent', 'MONEYLINE', 'CLOSING_SPREAD', 'CLOSING_TOTAL', 'Venue', 'Referee', 
           'ELO_Rating', 'Momentum',
             'HOME TEAM WIN%', 'HOME TEAM POINTS DIFFERENTIAL',
             'Opp_Elo', 'Opp_Momentum', 'CREW',
              'TOTAL POINTS PER GAME', 'CALLED FOULS PER GAME',
              'FOUL% AGAINST ROAD TEAMS', 'FOUL% AGAINST HOME TEAMS',
              'FOUL DIFFERENTIAL (Against Road Team) - (Against Home Team)',
              'SPREAD_LINE_MOVEMENT_1', 'SPREAD_LINE_MOVEMENT_2', 'SPREAD_LINE_MOVEMENT_3',
              'TOTAL_LINE_MOVEMENT_1', 'TOTAL_LINE_MOVEMENT_2', 'TOTAL_LINE_MOVEMENT_3']
# allow notebook to reload external modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
ml_model = BootstrapCalibratedClassifier(n_bootstrap_samples=5)
ml_model.load_model("ml_model")

spread_model = BootstrapCalibratedClassifier(n_bootstrap_samples=5)
spread_model.load_model("spread_model")

total_model = BootstrapCalibratedClassifier(n_bootstrap_samples=5)
total_model.load_model("total_model")


Models loaded from ml_model
Models loaded from spread_model
Models loaded from total_model


### today's features
- refs
- lines + movement
- today's games

In [181]:
current_odds = niu.scrape_odds()
ref_data = pd.read_csv('../historical_data/2024-2025.csv')

driver = niu.init_driver()
driver.get('https://official.nba.com/referee-assignments/')
time.sleep(5)
html_content = driver.page_source
driver.quit()


referee_data = niu.parse_referee_data(html_content)
for city, refs in referee_data.items():
    if city == 'L.A. Lakers':
        city = 'LA Lakers'
    try:
        current_odds[city][0] = ' '.join(refs[0].split(' ')[:-1])
        current_odds[city][-2] = ' '.join(refs[1].split(' ')[:-1])
        current_odds[city][-1] = ' '.join(refs[2].split(' ')[:-1])
    except KeyError:
        pass  # Handle any missing teams gracefully

#print(current_odds)
TODAY_MAP = niu.remove_ref_keys(current_odds)
TODAY_MAP = pd.DataFrame.from_dict(TODAY_MAP, orient='index', columns=[
    'Referee', 'MONEYLINE', 'Venue', 'Opponent', 'CLOSING_SPREAD', 'CLOSING_TOTAL', 'Spread_Movement', 'Total_Movement', 'CREW', 'CREW2'
])

today_teams_list = list(niu.team_map.values())
TODAY_MAP['TEAM'] = TODAY_MAP.index.copy(deep=True)
TODAY_MAP = TODAY_MAP.reset_index(drop=True)
TODAY_MAP = TODAY_MAP.merge(ref_data.groupby('REFEREE').first(), how='left', left_on='Referee', right_on='REFEREE', suffixes=['x', ''])
TODAY_MAP.tail()

scraping data for 2024-11-11


,Referee,MONEYLINE,Venue,Opponent,CLOSING_SPREAD,CLOSING_TOTAL,Spread_Movement,Total_Movement,CREW,CREW2,...,GENDER,EXPERIENCE (YEARS),GAMES OFFICIATED,HOME TEAM WIN%,HOME TEAM POINTS DIFFERENTIAL,TOTAL POINTS PER GAME,CALLED FOULS PER GAME,FOUL% AGAINST ROAD TEAMS,FOUL% AGAINST HOME TEAMS,FOUL DIFFERENTIAL (Against Road Team) - (Against Home Team)
0,Curtis Blair,-265,H,LA Clippers,-6.5,218.5,"[-6.5, -6.5, -6.5]","[218.5, 218.5, 218.5]",Aaron Smith,Jason Goldenberg,...,MALE,17,5,0.20,-9.6,224.4,41.0,0.483,0.517,-1.4
1,Curtis Blair,215,R,Oklahoma City,6.5,218.5,"[-6.5, -6.5, -6.5]","[218.5, 218.5, 218.5]",Aaron Smith,Jason Goldenberg,...,MALE,17,5,0.20,-9.6,224.4,41.0,0.483,0.517,-1.4
2,Jacyn Goble,-110,H,Brooklyn,1.5,210.0,"[-1.5, -1.5, -1.0]","[210.0, 210.5, 211.0]",Sean Corbin,Brandon Schwab,...,MALE,9,5,0.80,9.4,217.0,39.8,0.482,0.518,-1.4
3,Jacyn Goble,-120,R,New Orleans,-1.5,210.0,"[-1.5, -1.5, -1.0]","[210.0, 210.5, 211.0]",Sean Corbin,Brandon Schwab,...,MALE,9,5,0.80,9.4,217.0,39.8,0.482,0.518,-1.4
4,Brian Forte,-700,H,Washington,-12.0,229.5,"[-12.0, -12.5, -12.5]","[229.5, 229.5, 229.5]",Justin Van Duyne,Tyler Ricks,...,MALE,19,4,0.75,1.8,243.8,44.5,0.528,0.472,2.5
5,Brian Forte,500,R,Houston,12.0,229.5,"[-12.0, -12.5, -12.5]","[229.5, 229.5, 229.5]",Justin Van Duyne,Tyler Ricks,...,MALE,19,4,0.75,1.8,243.8,44.5,0.528,0.472,2.5
6,James Capers,270,H,Cleveland,8.5,237.0,"[-8.5, -8.5, -8.5]","[237.0, 237.0, 237.5]",Phenizee Ransom,Matt Myers,...,MALE,29,8,0.75,4.1,232.6,39.5,0.472,0.528,-2.3
7,James Capers,-340,R,Chicago,-8.5,237.0,"[-8.5, -8.5, -8.5]","[237.0, 237.0, 237.5]",Phenizee Ransom,Matt Myers,...,MALE,29,8,0.75,4.1,232.6,39.5,0.472,0.528,-2.3
8,Zach Zarba,-102,H,Sacramento,1.0,221.0,"[-1.0, -1.0, -1.0]","[221.0, 220.5, 220.5]",Ashley Moyer-Gleich,Brandon Adair,...,MALE,22,5,1.00,11.6,233.6,38.6,0.477,0.523,-1.8
9,Zach Zarba,-118,R,San Antonio,-1.0,221.0,"[-1.0, -1.0, -1.0]","[221.0, 220.5, 220.5]",Ashley Moyer-Gleich,Brandon Adair,...,MALE,22,5,1.00,11.6,233.6,38.6,0.477,0.523,-1.8


### most recent rows from historical dataset
- ELO scores
- Momentum scores


In [182]:
historical_data = pd.read_csv('2024_2025_nba_team_full.csv')
most_recent_historical = niu.get_most_recent_rows(historical_data, today_teams_list)
most_recent_historical[['TEAM', 'DATE', 'Elo_Rating', 'Momentum']].tail()

,TEAM,DATE,Elo_Rating,Momentum
14338,Atlanta,2024-11-06,1289.964062,-10.902854
14312,Brooklyn,2024-11-04,1358.553922,-2.404248
14340,Boston,2024-11-06,1442.710769,14.012092
14334,Charlotte,2024-11-06,1352.391861,-1.451689
14348,Chicago,2024-11-06,1399.647822,-2.846509
14346,Cleveland,2024-11-06,1434.189364,-8.228117
14349,Dallas,2024-11-06,1445.957167,-1.010324
14350,Denver,2024-11-06,1368.098635,-2.049594
14335,Detroit,2024-11-06,1423.992821,1.642776
14341,Golden State,2024-11-06,1454.896000,11.111465


### yesterday's data to update internal features
- running averages
- update ELO + momentum

In [183]:
yesterday_data = niu.download_current_data()
yesterday_df = pd.read_excel(yesterday_data)

# replace all white spaces in column names with _
yesterday_df.columns = yesterday_df.columns.str.replace(' ', '_')

# replace \n in column names with _
yesterday_df.columns = yesterday_df.columns.str.replace('\n', '_')
yesterday_df.columns = yesterday_df.columns.str.replace('__', '_')

yesterday_df['DATE'] = pd.to_datetime(yesterday_df['DATE'])
most_recent_historical_date = most_recent_historical['DATE'].max()
yesterday_df = yesterday_df[yesterday_df['DATE'] > most_recent_historical_date]
yesterday_df['Opponent'] = yesterday_df.groupby('GAME-ID')['TEAM'].shift(-1).fillna(yesterday_df.groupby('GAME-ID')['TEAM'].shift())

yesterday_df = yesterday_df.groupby('GAME-ID').apply(niu.assign_results)
yesterday_df[['DATE', 'TEAM', 'Opponent', 'PTS', 'GAME-ID', 'spread_result']].tail()

./11-10-2024-nba-season-team-feed.xlsx
200


C:\Users\chcro\AppData\Local\Temp\ipykernel_6888\2335341219.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  yesterday_df = yesterday_df.groupby('GAME-ID').apply(assign_results)


DATE       TEAM     Opponent  PTS   GAME-ID  spread_result
GAME-ID                                                                      
22400181 244 2024-11-07  Minnesota      Chicago  135  22400181           True
         245 2024-11-07    Chicago    Minnesota  119  22400181          False
22400182 242 2024-11-07       Utah    Milwaukee  100  22400182          False
         243 2024-11-07  Milwaukee         Utah  123  22400182           True
22400183 240 2024-11-07   Portland  San Antonio  105  22400183          False
...                 ...        ...          ...  ...       ...            ...
22400209 295 2024-11-10    Phoenix   Sacramento  118  22400209          False
22400210 298 2024-11-10    Memphis     Portland  134  22400210           True
         299 2024-11-10   Portland      Memphis   89  22400210          False
22400211 300 2024-11-10    Toronto    LA Lakers  103  22400211          False
         301 2024-11-10  LA Lakers      Toronto  123  22400211           True

[62 rows x 6 columns]

### Use most recent historical data as starting point to update ELO / Momentum

In [184]:
# most recent elo ratings and momentum scores from historical data
elo_ratings = most_recent_historical.set_index('TEAM')['Elo_Rating'].to_dict()
momentum_scores = most_recent_historical.set_index('TEAM')['Momentum'].to_dict()


# Iterate over the new data to update Elo and Momentum
for index, row in yesterday_df.iterrows():
    team_elo, team_momentum, opp_elo, opp_momentum = niu.update_elo_momentum(row, elo_ratings, momentum_scores)
    # Update the dictionaries with the new Elo and momentum values
    elo_ratings[row['TEAM']] = team_elo
    momentum_scores[row['TEAM']] = team_momentum
    elo_ratings[row['Opponent']] = opp_elo
    momentum_scores[row['Opponent']] = opp_momentum

# Output the updated Elo ratings and momentum scores
print("Updated Elo Ratings:", elo_ratings)
print("Updated Momentum Scores:", momentum_scores)


{'Atlanta': 1289.96406193833, 'Brooklyn': 1358.5539223121666, 'Boston': 1442.7107693608448, 'Charlotte': 1352.391861151739, 'Chicago': 1399.647822087457, 'Cleveland': 1434.189363894614, 'Dallas': 1445.9571670185553, 'Denver': 1368.098635117512, 'Detroit': 1423.9928206237655, 'Golden State': 1454.8959997463778, 'Houston': 1457.1232833810195, 'Indiana': 1451.1478748130305, 'LA Clippers': 1368.326178542249, 'LA Lakers': 1425.459460000944, 'Memphis': 1394.5285757513932, 'Miami': 1390.928410076334, 'Milwaukee': 1336.0001293809923, 'Minnesota': 1370.914225632382, 'New Orleans': 1354.0993495869077, 'New York': 1421.199746113635, 'Oklahoma City': 1481.8929174817922, 'Orlando': 1402.49114626816, 'Philadelphia': 1429.8452473701811, 'Phoenix': 1332.613085314924, 'Portland': 1408.192519451969, 'Sacramento': 1405.4933576557885, 'San Antonio': 1472.9833468571123, 'Toronto': 1412.7555106843945, 'Utah': 1370.28178422329, 'Washington': 1345.6247613905855}
Updated Elo Ratings: {'Atlanta': 1296.257843250

### Update TODAY_MAP with new elo/momentum (opps too)

In [186]:
# convert index to TEAM column in TODAY_MAP

TODAY_MAP['ELO_Rating'] = TODAY_MAP['TEAM'].map(elo_ratings)
TODAY_MAP['Momentum'] = TODAY_MAP['TEAM'].map(momentum_scores)
TODAY_MAP['Opp_Elo'] = TODAY_MAP['Opponent'].map(elo_ratings)
TODAY_MAP[['SPREAD_LINE_MOVEMENT_1', 'SPREAD_LINE_MOVEMENT_2', 'SPREAD_LINE_MOVEMENT_3']] = pd.DataFrame(TODAY_MAP['Spread_Movement'].to_list(), index=TODAY_MAP.index)
TODAY_MAP[['TOTAL_LINE_MOVEMENT_1', 'TOTAL_LINE_MOVEMENT_2', 'TOTAL_LINE_MOVEMENT_3']] = pd.DataFrame(TODAY_MAP['Total_Movement'].to_list(), index=TODAY_MAP.index)
TODAY_MAP[['OPP_SPREAD_MOVEMENT_1', 'OPP_SPREAD_MOVEMENT_2', 'OPP_SPREAD_MOVEMENT_3']] = TODAY_MAP['Opponent'].map(TODAY_MAP.set_index('TEAM')[['SPREAD_LINE_MOVEMENT_1', 'SPREAD_LINE_MOVEMENT_2', 'SPREAD_LINE_MOVEMENT_3']].to_dict('index')).apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series([None, None, None]))
TODAY_MAP[['OPP_TOTAL_MOVEMENT_1', 'OPP_TOTAL_MOVEMENT_2', 'OPP_TOTAL_MOVEMENT_3']] = TODAY_MAP['Opponent'].map(TODAY_MAP.set_index('TEAM')[['TOTAL_LINE_MOVEMENT_1', 'TOTAL_LINE_MOVEMENT_2', 'TOTAL_LINE_MOVEMENT_3']].to_dict('index')).apply(lambda x: pd.Series(x) if isinstance(x, dict) else pd.Series([None, None, None]))

TODAY_MAP['Opp_Momentum'] = TODAY_MAP['Opponent'].map(momentum_scores)
TODAY_MAP[['TEAM', 'Opponent', 'MONEYLINE', 'CLOSING_SPREAD', 'CLOSING_TOTAL']]



,TEAM,Opponent,MONEYLINE,CLOSING_SPREAD,CLOSING_TOTAL
0,Oklahoma City,LA Clippers,-265,-6.5,218.5
1,LA Clippers,Oklahoma City,215,6.5,218.5
2,New Orleans,Brooklyn,-110,1.5,210.0
3,Brooklyn,New Orleans,-120,-1.5,210.0
4,Houston,Washington,-700,-12.0,229.5
5,Washington,Houston,500,12.0,229.5
6,Chicago,Cleveland,270,8.5,237.0
7,Cleveland,Chicago,-340,-8.5,237.0
8,San Antonio,Sacramento,-102,1.0,221.0
9,Sacramento,San Antonio,-118,-1.0,221.0


### update rolling stats, collect features, and perform inference

In [207]:
# get all the rolling stats for today
most_recent_tdf = niu.get_rolling_stats(yesterday_df, today_teams_list)

today_features = TODAY_MAP[today_map_features]

# merge most_recent_tdf with today_features on TEAM
infer_df = most_recent_tdf.merge(today_features, how='left', on='TEAM')

# convert categorical columns
infer_df['MAIN REF'] = infer_df['Referee'].astype('category')
infer_df['TEAM'] = infer_df['TEAM'].astype('category')
infer_df['CREW'] = infer_df['CREW'].astype('category')
infer_df['Opponent'] = infer_df['Opponent_y'].astype('category')
infer_df['TEAM_REST_DAYS'] = infer_df['TEAM_REST_DAYS'].astype('category')

# fix merged column names
infer_df['MONEYLINE'] = infer_df['MONEYLINE_y']
infer_df['VENUE'] = infer_df['Venue']
infer_df['CLOSING_SPREAD'] = infer_df['CLOSING_SPREAD_y']
infer_df['CLOSING_TOTAL'] = infer_df['CLOSING_TOTAL_y']
infer_df['Elo_Rating'] = infer_df['ELO_Rating']

# convert datatypes
infer_df['VENUE'] = (infer_df['VENUE'] == 'H')*1
infer_df[["MONEYLINE", "Last_ML_1", "Last_ML_2", "Last_ML_3"]] = (
        infer_df[["MONEYLINE", "Last_ML_1", "Last_ML_2", "Last_ML_3"]]
        .replace('even', '-100', regex=True)
        .fillna(0)
        .astype(int)
    )

# filter down to train cols
infer_df = infer_df[train_cols]

# Get predictions from the ensemble models
spread_probabilities = spread_model.predict_proba(infer_df)[:, 1]
ml_probabilities = ml_model.predict_proba(infer_df)[:, 1]
total_probabilities = total_model.predict_proba(infer_df)[:, 1]
spread_predictions = np.array([x > 0.5 for x in spread_probabilities])
ml_predictions = np.array([x > 0.5 for x in ml_probabilities])
total_predictions = np.array([x > 0.5 for x in total_probabilities])

infer_df['spread_prob'] = spread_probabilities
infer_df['ml_prob'] = ml_probabilities
infer_df['total_prob'] = total_probabilities

# get the results
today_results = infer_df[['TEAM', 'Opponent', 'MONEYLINE',
                        'CLOSING_SPREAD', 'CLOSING_TOTAL',
                        'spread_prob', 'ml_prob', 'total_prob']].dropna().reset_index(drop=True)

spread_ps = {team: prob for team, prob in zip(today_results['TEAM'].values, spread_probabilities)}
normed_spread_odds = {team: spread_ps[team]/(spread_ps[team] + spread_ps[opp]) for team, opp in zip(today_results['TEAM'], today_results['Opponent'])}
ml_ps = {team: prob for team, prob in zip(today_results['TEAM'].values, ml_probabilities)}
normed_ml_odds = {team: ml_ps[team]/(ml_ps[team] + ml_ps[opp]) for team, opp in zip(today_results['TEAM'], today_results['Opponent'])}
total_ps = {team: prob for team, prob in zip(today_results['TEAM'].values, total_probabilities)}
normed_total_odds = {team: total_ps[team]/(total_ps[team] + total_ps[opp]) for team, opp in zip(today_results['TEAM'], today_results['Opponent'])}

today_results['spread_prob_normed'] = today_results['TEAM'].map(normed_spread_odds)
today_results['ml_prob_normed'] = today_results['TEAM'].map(normed_ml_odds)
today_results['total_prob_normed'] = today_results['TEAM'].map(normed_total_odds)


### print spread predictions

In [202]:
print('NORMALIZED PREDICTED WINNERS\n')
display(today_results[(today_results['spread_prob_normed'] > 0.5) & (today_results['spread_prob'] > 0.5)] \
.sort_values('spread_prob', ascending=False)[['TEAM', 'Opponent','CLOSING_SPREAD',
                                                'spread_prob', 'spread_prob_normed']])

print('\nALL PREDICTED WINNERS\n')
today_results[(today_results['spread_prob_normed'] > 0.5)] \
.sort_values('spread_prob', ascending=False)[['TEAM', 'Opponent','CLOSING_SPREAD',
                                                'spread_prob', 'spread_prob_normed']]


NORMALIZED PREDICTED WINNERS



,TEAM,Opponent,CLOSING_SPREAD,spread_prob,spread_prob_normed
6,Oklahoma City,LA Clippers,-6.5,0.830927,0.792339
7,Sacramento,San Antonio,-1.0,0.638201,0.552570



ALL PREDICTED WINNERS



,TEAM,Opponent,CLOSING_SPREAD,spread_prob,spread_prob_normed
6,Oklahoma City,LA Clippers,-6.5,0.830927,0.792339
7,Sacramento,San Antonio,-1.0,0.638201,0.552570
2,Cleveland,Chicago,-8.5,0.313974,0.743817
0,Brooklyn,New Orleans,-1.5,0.295606,0.657303
9,Washington,Houston,12.0,0.091843,0.567577


### print ml predictions

In [203]:
print('NORMALIZED PREDICTED WINNERS\n')
display(today_results[(today_results['ml_prob_normed'] > 0.5) & (today_results['ml_prob'] > 0.5)] \
.sort_values('ml_prob', ascending=False)[['TEAM', 'Opponent','MONEYLINE',
                                                'ml_prob', 'ml_prob_normed']])

print('ALL PREDICTED WINNERS\n')
display(today_results[(today_results['ml_prob_normed'] > 0.5)] \
.sort_values('ml_prob', ascending=False)[['TEAM', 'Opponent','MONEYLINE',
                                                'ml_prob', 'ml_prob_normed']])


NORMALIZED PREDICTED WINNERS



,TEAM,Opponent,MONEYLINE,ml_prob,ml_prob_normed
6,Oklahoma City,LA Clippers,-265,0.69374,0.5656


ALL PREDICTED WINNERS



,TEAM,Opponent,MONEYLINE,ml_prob,ml_prob_normed
6,Oklahoma City,LA Clippers,-265,0.693740,0.565600
7,Sacramento,San Antonio,-118,0.320288,0.565467
1,Chicago,Cleveland,270,0.282341,0.513657
5,New Orleans,Brooklyn,-110,0.195045,0.772991
9,Washington,Houston,500,0.086804,0.588468


### print total predictions

In [211]:
print('PREDICTED OVERS')
display(today_results[(today_results['total_prob'] > 0.5)] \
.sort_values('total_prob', ascending=False)[['TEAM', 'Opponent','CLOSING_TOTAL',
                                                'total_prob', 'total_prob_normed']])

print('\nPREDICTED UNDERS\n')
display(today_results[(today_results['total_prob'] <= 0.5)] \
.sort_values('total_prob', ascending=False)[['TEAM', 'Opponent','CLOSING_TOTAL',
                                                'total_prob', 'total_prob_normed']])

PREDICTED OVERS


,TEAM,Opponent,CLOSING_TOTAL,total_prob,total_prob_normed



PREDICTED UNDERS



,TEAM,Opponent,CLOSING_TOTAL,total_prob,total_prob_normed
8,San Antonio,Sacramento,221.0,0.452294,0.486139
1,Chicago,Cleveland,237.0,0.416558,0.166295
6,Oklahoma City,LA Clippers,218.5,0.378215,0.691814
9,Washington,Houston,229.5,0.308397,0.585284
3,Houston,Washington,229.5,0.291042,0.414716
7,Sacramento,San Antonio,221.0,0.258166,0.513861
2,Cleveland,Chicago,237.0,0.177529,0.833705
5,New Orleans,Brooklyn,210.0,0.137623,0.152498
0,Brooklyn,New Orleans,210.0,0.123653,0.847502
4,LA Clippers,Oklahoma City,218.5,0.059990,0.308186
